In [28]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.messages.system import SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
import numexpr
import math

In [29]:
@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

In [30]:
tools = [calculator]

model = ChatOllama(model="llama3.2:latest").bind_tools(tools)

In [43]:
memory = MemorySaver()

sys_msg = SystemMessage("You can use the tools you are provided with to solve problems. "
                        "If you think the tools are not necessary, just answer the question directly.")

agent = create_react_agent(model, tools, checkpointer=memory, state_modifier=sys_msg)

In [45]:
config = {"configurable": {"thread_id": "def234"}}

input_message = (
    "What's 3.2 elevated to the power of 0.51?"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What's 3.2 elevated to the power of 0.51?
================================== Ai Message ==================================
Tool Calls:
  calculator (c9bb0fb1-136d-46ba-9ffd-76f8b9dbdb44)
 Call ID: c9bb0fb1-136d-46ba-9ffd-76f8b9dbdb44
  Args:
    expression: 3.2**(0.51)
================================= Tool Message =================================
Name: calculator

1.8097829355956991
================================== Ai Message ==================================

The result of 3.2 elevated to the power of 0.51 is approximately 1.8097829355956991.


In [ ]:
input_message = ("Hi llama")

for event in agent.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'checkpoint_ns', 'checkpoint_id']

In [42]:
messages

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2024-12-02T15:52:44.957226Z', 'done': True, 'done_reason': 'stop', 'total_duration': 392228000, 'load_duration': 16410300, 'prompt_eval_count': 217, 'prompt_eval_duration': 4000000, 'eval_count': 17, 'eval_duration': 371000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='calculator', arguments={'expression': 'Hello world'}))])}, id='run-cae206b6-35e8-4c9c-bbd4-48527fd4d647-0', tool_calls=[{'name': 'calculator', 'args': {'expression': 'Hello world'}, 'id': '18a506b4-2c45-40f5-8a82-e929d3c50d0a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 17, 'total_tokens': 234})